In [58]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [59]:
from skfeature.function.similarity_based import reliefF
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_tongren.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

In [60]:
techini_indicator=data_rfe.columns.tolist()[:-1]

# SVC

In [61]:
estimator_SVC=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVC.fit(x_train,y_train)
SVC_result= estimator_SVC.predict(x_test)
performan_SVC=meansure_performance(SVC_result)
print(performan_SVC)

(0.7823529411764706, 0.810126582278481, 0.7582417582417582, 0.7441860465116279, 0.7757575757575758)


 # CFS-SVM

In [62]:
CFS_selected_feature = CFS.cfs(x_train,y_train)
CFS_fea_sele=list(CFS_selected_feature)
CFS_fea_sele

[26, 27, 8, 31, 1, 2]

In [63]:
np.array(techini_indicator)[CFS_fea_sele],len(np.array(techini_indicator)[CFS_fea_sele])

(array(['K', 'D', 'VR', 'VROC', 'TAPI', 'VMA'], dtype='<U8'), 6)

In [64]:
estimator_SVM_CFS=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVM_CFS.fit(x_train[:,CFS_fea_sele],y_train)
CFS_SVM_result=estimator_SVM_CFS.predict(x_test[:,CFS_fea_sele])

In [65]:
performan_CFS_SVM=meansure_performance(CFS_SVM_result)
print(performan_CFS_SVM)

(0.6823529411764706, 0.569620253164557, 0.7802197802197802, 0.6923076923076923, 0.625)


# sequencial-backward-elimination SVM

In [66]:

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class back_SVM:
    def __init__(self,X_train,Y_train,c,gamma):
        self.X_train=X_train
        self.Y_train=Y_train
        self.C=c
        self.Gamma=gamma
    def SBE(self,folds):
        fea_num=self.X_train.shape[1]
        candi_fea=[i for i in range(fea_num)]
        fea=[i for i in range(fea_num)]
        best_subset=[i for i in range(fea_num)]
        model=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        Best_accuracy=mean(cross_val_score(model,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        Success=True
        while Success:
            Success=False
            for f in candi_fea:
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(f)
                x_sele=self.X_train[:,candi_fea_copy]
                n_scores=cross_val_score(model,x_sele,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1)
                acc=mean(n_scores)
                if acc>=Best_accuracy:
                    #print(f)
                    best_subset=candi_fea_copy
                    Success=True
                    Best_accuracy=acc
            print(1)
            if Success==True:
                candi_fea=best_subset
            print(len(candi_fea))
        return best_subset
                

In [67]:
estimator_SVM_back=back_SVM(x_train,y_train,1.5,0.1)
candi_fea_SBE=estimator_SVM_back.SBE(5)

1
48
1
47
1
46
1
45
1
44
1
43
1
42
1
41
1
40
1
39
1
38
1
37
1
36
1
35
1
35


In [68]:
np.array(techini_indicator)[candi_fea_SBE],len(np.array(techini_indicator)[candi_fea_SBE])

(array(['VMA', 'VMACD', 'CR', 'PSY', 'BR', 'WAD', 'MOBV', 'PVT', 'MA',
        'MACD', 'MTM', 'PRICEOSC', 'BIAS', 'CCI', 'DBCD', 'DPO', 'K', 'D',
        'J', 'RSI', 'SRDM', 'VROC', 'VRSI', 'WR', 'ADTM', 'DPTB', 'JDQS',
        'JDRS', 'MI', 'RCCD', 'SRMI', 'ATR', 'MASS', 'STD', 'VHF'],
       dtype='<U8'),
 35)

In [69]:
estimator_SVM_SBE=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVM_SBE.fit(x_train[:,candi_fea_SBE],y_train)
SBE_SVM_result=estimator_SVM_SBE.predict(x_test[:,candi_fea_SBE])
performan_SBE_SVM=meansure_performance(SBE_SVM_result)
print(performan_SBE_SVM)

(0.8176470588235294, 0.8354430379746836, 0.8021978021978022, 0.7857142857142857, 0.8098159509202455)


In [82]:
candi_fea_SBE

[2,
 3,
 6,
 7,
 10,
 11,
 12,
 13,
 16,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 42,
 43,
 44,
 46,
 47,
 48]

In [96]:
np.array(techini_indicator)[45]

'CVLT'

# reliefF-svm 

In [70]:
from skfeature.function.similarity_based import reliefF
relief_fea_score=reliefF.reliefF(x_train,y_train)
relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))[:14]

In [71]:
relief_fea_score

array([ 0.06245391,  0.03024295,  0.00132751,  0.01251561, -0.00800629,
       -0.00517335, -0.01409416,  0.00958632, -0.00878644, -0.01555002,
       -0.01226751, -0.00446239,  0.50911864, -0.00691375, -0.00087673,
       -0.003301  , -0.00193329, -0.00487132, -0.00777415,  0.00366475,
        0.00622239,  0.00454909,  0.03821541, -0.00154434, -0.00690483,
        0.04466628, -0.01044506,  0.01239301, -0.00988858,  0.0795882 ,
       -0.0007369 ,  0.01771665,  0.07222418,  0.06201413, -0.00853768,
        0.00548758,  0.00188326, -0.00745072, -0.00690156, -0.00113523,
        0.00121344, -0.01059238, -0.01337206,  0.0097242 , -0.00241915,
       -0.00768529, -0.0236413 ,  0.00241743, -0.00768038])

In [72]:
np.array(techini_indicator)[relief_candi_fea],len(np.array(techini_indicator)[relief_candi_fea])

(array(['MOBV', 'RSI', 'VRSI', 'QRR', 'WR', 'DPO', 'BIAS', 'TAPI', 'VROC',
        'VMACD', 'D', 'SRMI', 'PSY', 'PRICEOSC'], dtype='<U8'),
 14)

In [73]:
estimator_SVM_relief=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVM_relief.fit(x_train[:,relief_candi_fea],y_train)
relief_SVM_result=estimator_SVM_relief.predict(x_test[:,relief_candi_fea])
performan_relief_SVM=meansure_performance(relief_SVM_result)
print(performan_relief_SVM)

(0.8176470588235294, 0.7974683544303798, 0.8351648351648352, 0.8076923076923077, 0.802547770700637)


# RFE-SVM

In [74]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [75]:
rfe_SVM_model=rfe_SVM(x_train,y_train,1.5,0.1)
rfe_SVM_sele_featu_subset=rfe_SVM_model.featu_subset(14)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [76]:
np.array(techini_indicator)[rfe_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_SVM_sele_featu_subset])

(array(['VSTD', 'PSY', 'MOBV', 'MTM', 'CCI', 'K', 'D', 'J', 'RSI', 'SRDM',
        'VRSI', 'WR', 'MICD', 'SRMI'], dtype='<U8'),
 14)

In [77]:
estimator_SVM_rfe=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVM_rfe.fit(x_train[:,rfe_SVM_sele_featu_subset],y_train)
rfe_SVM_result=estimator_SVM_rfe.predict(x_test[:,rfe_SVM_sele_featu_subset])
performan_rfe_SVM=meansure_performance(rfe_SVM_result)
print(performan_rfe_SVM)

(0.8235294117647058, 0.7848101265822784, 0.8571428571428571, 0.8266666666666667, 0.8051948051948051)


# SVM-RFE_RELIEF

In [78]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1
            

In [79]:
# k=4# k=5, 0.8
rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,1.5,0.1)
rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.55,14)

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15


In [80]:
np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset])

(array(['VSTD', 'PSY', 'MOBV', 'MTM', 'CCI', 'K', 'D', 'J', 'RSI', 'SRDM',
        'VRSI', 'WR', 'MICD', 'SRMI'], dtype='<U8'),
 14)

In [81]:
estimator_SVM_rfe_reliefF=SVC(C=1.5, kernel='rbf',gamma=0.1)
estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
print(performan_rfe_reliefF_SVM)

(0.8235294117647058, 0.7848101265822784, 0.8571428571428571, 0.8266666666666667, 0.8051948051948051)
